<a href="https://colab.research.google.com/github/solong96/solong96/blob/main/0818_cc_text_classfication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers.rnn import LSTM, GRU,SimpleRNN
from keras.layers.core import Dense, Activation, Dropout, Embedding
from keras.layers.normalization import batch_normalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.utils import pad_sequences
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping


import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
import os

In [3]:
path = '/content/drive/MyDrive/0809/'
os.chdir(path)

In [4]:
index_list = list(range(0,10,1))
word = pd.DataFrame({'cate':["energy","industry","waste","traffic","building","land-use","green","biodiversity","water","heat island"],
                                    'toxic':[11,12,13,14,15,16,21,22,23,24],
                                    'class':index_list})
word

,cate,toxic,class
0,energy,11,0
1,industry,12,1
2,waste,13,2
3,traffic,14,3
4,building,15,4
5,land-use,16,5
6,green,21,6
7,biodiversity,22,7
8,water,23,8
9,heat island,24,9


In [5]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  grpc://10.63.13.130:8470
REPLICAS:  8


In [6]:
# train_dataset = pd.read_excel('0818_cc_train_token_title.xlsx')
train_dataset = pd.read_excel('0818_cc_train_token_abstract2.xlsx')
train_dataset

,Unnamed: 0,Abstract,token,category,toxic
0,0,The urban heat island effect (UHI) is a produc...,urban heat-island UHU product urbanization cer...,green,21
1,1,Global warming is increasing extreme heat cond...,global extreme heat condition energy efficienc...,energy,11
2,2,Renewable energy is considered crucial for cli...,renewable energy crucial climate-change mitiga...,energy,11
3,3,Coastal groundwater-dependent ecosystems (GDEs...,coastal groundwater-dependent ecosystem gde we...,water,23
4,4,Reductions in carbon emissions have been a foc...,reduction carbon-emission focus power sector s...,energy,11
...,...,...,...,...,...
13188,13188,Electrochemical batteries are essential for re...,electrochemical battery essential renewable su...,energy,11
13189,13189,Many cities across the United States have turn...,many city united state energy disclosure bench...,energy,11
13190,13190,There is a compelling body of evidence calling...,body reduction amount carbon dioxide CO2 manki...,energy,11
13191,13191,This paper presents a multi-criteria optimizat...,present multi-criterion optimization power sys...,energy,11


In [7]:
train = train_dataset[['token','toxic']]
train

,token,toxic
0,urban heat-island UHU product urbanization cer...,21
1,global extreme heat condition energy efficienc...,11
2,renewable energy crucial climate-change mitiga...,11
3,coastal groundwater-dependent ecosystem gde we...,23
4,reduction carbon-emission focus power sector s...,11
...,...,...
13188,electrochemical battery essential renewable su...,11
13189,many city united state energy disclosure bench...,11
13190,body reduction amount carbon dioxide CO2 manki...,11
13191,present multi-criterion optimization power sys...,11


In [8]:
# print(train['token'].apply(lambda x:len(str(x).split())).max())
# max_len = (round(train['token'].apply(lambda x:len(str(x).split())).max()/100)*100)+50
# print(max_len)

1034
1050


In [9]:
xtrain, xvalid, y_train, y_valid = train_test_split(train.token.values, train.toxic.values,
                                                  stratify=train.toxic.values,
                                                  random_state=42,
                                                  test_size=0.2, shuffle=True)

In [10]:
# using keras tokenizer here
token = text.Tokenizer(num_words=None)
# max_len = 1000

token.fit_on_texts(list(xtrain) + list(xvalid))
xtrain_seq = token.texts_to_sequences(xtrain)
xvalid_seq = token.texts_to_sequences(xvalid)

#zero pad the sequences
xtrain_pad = pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = pad_sequences(xvalid_seq, maxlen=max_len)

word_index = token.word_index

In [11]:
# Calculating the Vocabulary Size and the number of Answers

print("Vocabulary size={}".format(len(token.word_index)))
print("Number of Answers={}".format(token.document_count))

Vocabulary size=27678
Number of Answers=13193


In [ ]:
max_len = max(len(l) for l in xtrain_seq)
print('문장의 최대 길이 :', max_len)

In [12]:
le = LabelEncoder()
num_classes=10

# 0          energy
# 1        industry
# 2           waste
# 3         traffic
# 4        building
# 5        land-use
# 6           green
# 7    biodiversity
# 8           water
# 9     heat island

ytrain = le.fit_transform(y_train)
yvalid = le.transform(y_valid)

ytrain = to_categorical(ytrain)
yvalid = to_categorical(yvalid)

In [13]:
ytrain

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [14]:
ytrain.shape

(10554, 10)

# Word Embeddings

While building our simple RNN models we talked about using word-embeddings , So what is word-embeddings and how do we get word-embeddings?
Here is the answer :
* https://www.coursera.org/learn/nlp-sequence-models/lecture/6Oq70/word-representation
* https://machinelearningmastery.com/what-are-word-embeddings/
<br> <br>
The latest approach to getting word Embeddings is using pretained GLoVe or using Fasttext. Without going into too much details, I would explain how to create sentence vectors and how can we use them to create a machine learning model on top of it and since I am a fan of GloVe vectors, word2vec and fasttext. In this Notebook, I'll be using the GloVe vectors. You can download the GloVe vectors from here http://www-nlp.stanford.edu/data/glove.840B.300d.zip or you can search for GloVe in datasets on Kaggle and add the file

In [15]:
# load the GloVe vectors in a dictionary:

embeddings_index = {}
f = open('glove.6B.300d.txt','r',encoding='utf-8')
for line in tqdm(f):
    values = line.split(' ')
    word = values[0]
    coefs = np.asarray([float(val) for val in values[1:]])
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

400001it [01:14, 5374.25it/s]

Found 400001 word vectors.


1D CNN

In [16]:
# create an embedding matrix for the words we have in the dataset
embedding_matrix = np.zeros((len(word_index) + 1, 300)) # 300은 차원 수
for word, i in tqdm(word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

100%|██████████| 27678/27678 [00:00<00:00, 39573.41it/s]


In [19]:
dict(zip(list(le.classes_), le.transform(list(le.classes_))))

{11: 0, 12: 1, 13: 2, 14: 3, 15: 4, 16: 5, 21: 6, 22: 7, 23: 8, 24: 9}

In [20]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding, Dropout, Conv1D, GlobalMaxPooling1D, Dense, Input, Flatten, Concatenate

kernel_sizes = [2, 3, 5]
num_filters = 512
dropout_ratio = 0.5

vocab_size = len(word_index) + 1
label_idx = dict(zip(list(le.classes_), le.transform(list(le.classes_))))

model_input = Input(shape=(max_len,))
output = Embedding(vocab_size, 300, weights=[embedding_matrix],
                   input_length=max_len, trainable=False)(model_input)

conv_blocks = []

for size in kernel_sizes:
    conv = Conv1D(filters=num_filters,
                         kernel_size=size,
                         padding="valid",
                         activation="relu",
                         strides=1)(output)
    conv = GlobalMaxPooling1D()(conv)
    conv_blocks.append(conv)

output = Concatenate()(conv_blocks) if len(conv_blocks) > 1 else conv_blocks[0]
output = Dropout(dropout_ratio)(output)
model_output = Dense(len(label_idx), activation='softmax')(output)
model = Model(model_input, model_output)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1050)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, 1050, 300)    8303700     ['input_1[0][0]']                
                                                                                                  
 conv1d (Conv1D)                (None, 1049, 512)    307712      ['embedding[0][0]']              
                                                                                                  
 conv1d_1 (Conv1D)              (None, 1048, 512)    461312      ['embedding[0][0]']              
                                                                                              

In [ ]:
history = model.fit(xtrain_pad, ytrain,
          batch_size=64*strategy.num_replicas_in_sync,
          epochs=50,
          validation_data=(xvalid_pad, yvalid))

Epoch 1/50
21/21 [==============================] - 120s 6s/step - loss: 1.1751 - acc: 0.7007 - val_loss: 0.4704 - val_acc: 0.8613
Epoch 2/50
21/21 [==============================] - 117s 6s/step - loss: 0.4354 - acc: 0.8605 - val_loss: 0.3261 - val_acc: 0.8878
Epoch 3/50
21/21 [==============================] - 118s 6s/step - loss: 0.2841 - acc: 0.9016 - val_loss: 0.2936 - val_acc: 0.8897
Epoch 4/50


In [ ]:
epochs = range(1, len(history.history['acc']) + 1)
plt.plot(epochs, history.history['acc'])
plt.plot(epochs, history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epochs')
plt.legend(['train', 'test'], loc='lower right')
plt.show()

epochs = range(1, len(history.history['loss']) + 1)
plt.plot(epochs, history.history['loss'])
plt.plot(epochs, history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epochs')
plt.legend(['train', 'test'], loc='upper right')
plt.show()

In [ ]:
# using keras tokenizer here
token = text.Tokenizer(num_words=None)
# max_len = 1000

token.fit_on_texts(list(xtrain) + list(xvalid))
xtrain_seq = token.texts_to_sequences(xtrain)


X_test = tokenizer.texts_to_sequences(X_test)
X_test = pad_sequences(X_test, maxlen=max_len)

In [ ]:
y_predicted = model.predict(X_test)
y_predicted = y_predicted.argmax(axis=-1) # 예측을 정수 시퀀스로 변환

In [ ]:
print('정확도(Accuracy) : ', sum(y_predicted == y_test) / len(y_test))

# LSTM's

## Basic Overview

Simple RNN's were certainly better than classical ML algorithms and gave state of the art results, but it failed to capture long term dependencies that is present in sentences . So in 1998-99 LSTM's were introduced to counter to these drawbacks.

## In Depth Understanding

Why LSTM's?
* https://www.coursera.org/learn/nlp-sequence-models/lecture/PKMRR/vanishing-gradients-with-rnns
* https://www.analyticsvidhya.com/blog/2017/12/fundamentals-of-deep-learning-introduction-to-lstm/

What are LSTM's?
* https://www.coursera.org/learn/nlp-sequence-models/lecture/KXoay/long-short-term-memory-lstm
* https://distill.pub/2019/memorization-in-rnns/
* https://towardsdatascience.com/illustrated-guide-to-lstms-and-gru-s-a-step-by-step-explanation-44e9eb85bf21

# Code Implementation

We have already tokenized and paded our text for input to LSTM's

In [ ]:
# create an embedding matrix for the words we have in the dataset
embedding_matrix = np.zeros((len(word_index) + 1, 300)) # 300은 차원 수
for word, i in tqdm(word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

100%|██████████| 27678/27678 [00:00<00:00, 131120.26it/s]


LSTM(n, input_dim = k, input_length = k)

n : 메모리 셀의 개수(기억용량, 출력형태를 결정함)

In [ ]:
# %%time
with strategy.scope():

    # A simple LSTM with glove embeddings and one dense layer
    model = Sequential()
    model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))

    model.add(LSTM(128, dropout=0.3, recurrent_dropout=0.3))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 1050, 300)         8303700   
                                                                 
 lstm_3 (LSTM)               (None, 128)               219648    
                                                                 
 dense_6 (Dense)             (None, 64)                8256      
                                                                 
 dense_7 (Dense)             (None, 10)                650       
                                                                 
Total params: 8,532,254
Trainable params: 228,554
Non-trainable params: 8,303,700
_________________________________________________________________


In [ ]:
model.fit(xtrain_pad, ytrain, validation_data = (xvalid_pad, yvalid), epochs=50, batch_size=64*strategy.num_replicas_in_sync)

Epoch 1/50
21/21 [==============================] - 17s 508ms/step - loss: 1.5385 - accuracy: 0.5482 - val_loss: 1.1825 - val_accuracy: 0.6048
Epoch 2/50
21/21 [==============================] - 6s 278ms/step - loss: 0.9394 - accuracy: 0.7055 - val_loss: 0.6734 - val_accuracy: 0.8102
Epoch 3/50
21/21 [==============================] - 4s 209ms/step - loss: 0.6220 - accuracy: 0.8114 - val_loss: 0.5362 - val_accuracy: 0.8325
Epoch 4/50
21/21 [==============================] - 4s 191ms/step - loss: 0.4777 - accuracy: 0.8580 - val_loss: 0.4985 - val_accuracy: 0.8609
Epoch 5/50
21/21 [==============================] - 4s 189ms/step - loss: 0.3870 - accuracy: 0.8852 - val_loss: 0.3910 - val_accuracy: 0.8825
Epoch 6/50
21/21 [==============================] - 4s 199ms/step - loss: 0.3177 - accuracy: 0.9022 - val_loss: 0.3275 - val_accuracy: 0.8958
Epoch 7/50
21/21 [==============================] - 5s 243ms/step - loss: 0.2794 - accuracy: 0.9156 - val_loss: 0.3038 - val_accuracy: 0.9022
Epoch

In [ ]:
scores = model.evaluate(xvalid_pad, yvalid)
print("Accuracy: %.2f%%" % (scores[1]*100))

83/83 [==============================] - 6s 51ms/step - loss: 0.2392 - accuracy: 0.9447
Accuracy: 94.47%


In [ ]:
model.save('10_LSTM_abs.h5', save_format='h5')
model.save('10_LSTM_abs.keras')

In [ ]:
import pickle
pickle.dump(model, open('10_LSTM_abs.pkl', 'wb'))

In [ ]:
yvalid_model = model.predict(xvalid_pad)
yvalid_model

83/83 [==============================] - 5s 49ms/step


array([[9.99980807e-01, 2.51202391e-06, 1.10840278e-06, ...,
        1.62333791e-07, 6.68762141e-06, 3.22537608e-06],
       [6.57662099e-07, 7.02152374e-06, 3.05019921e-06, ...,
        2.57396646e-08, 5.02509351e-07, 5.01719398e-07],
       [9.99926925e-01, 5.08488938e-07, 2.00429758e-06, ...,
        7.17102807e-07, 1.85106603e-06, 1.02507893e-05],
       ...,
       [9.99999046e-01, 1.37716299e-07, 4.99800450e-08, ...,
        7.22773086e-09, 3.49454950e-07, 1.04139104e-07],
       [7.77836613e-05, 6.28296129e-06, 8.10446821e-08, ...,
        1.92460092e-07, 1.10601445e-06, 6.21666345e-07],
       [8.53542861e-06, 3.48234028e-02, 1.94189797e-05, ...,
        3.61421861e-08, 2.68057897e-06, 1.41886153e-06]], dtype=float32)

0	energy
1	industry
2	waste
3	traffic
4	building
5	land-use
6	green
7	biodiversity
8	water
9	heat island

In [ ]:
#converting the one hot vector output to a linear numpy array.
yvalid_class = np.argmax(yvalid_model, axis = 1)

In [ ]:
word[(word['toxic'] == y_valid[1])]['class'].values[0]

3

In [ ]:
y_valid_class = []
for i in range(len(y_valid)):
  y_valid_class.append(word[(word['toxic'] == y_valid[i])]['class'].values[0])

In [ ]:
y_valid[1], y_valid_class[1]

(14, 3)

In [ ]:
#extracting the classes from the label encoder
encoded_classes = le.classes_

#mapping the encoded output to actual categories
predicted_category = [encoded_classes[x] for x in yvalid_class]
true_category = [encoded_classes[x] for x in y_valid_class]

In [ ]:
result_df = pd.DataFrame({'description':xvalid,'true_category':true_category, 'predicted_category':predicted_category})
result_df.head()

,description,true_category,predicted_category
0,climate-change concerned environmental issue w...,11,11
1,main challenge transportation industry labor c...,14,14
2,study variance overall greenhouse gas mitigati...,11,11
3,various definition energy net-zero energy buil...,11,11
4,energy inevitable necessity human energy sourc...,11,11


In [ ]:
result_df.to_excel('0818_cc_lstm_result.xlsx')

In [ ]:
digital = pd.read_excel('0818_digital_train_token_abstract.xlsx')
digital

,Unnamed: 0,Abstract,token,category
0,0,Technological advancements such as information...,technological advancement information communic...,energy
1,2,The Internet of Things (IoT) has become a hot ...,internet thing iot hot topic present tech-driv...,traffic
2,6,Technological innovation is constantly reshapi...,technological innovation materiality mechanic ...,traffic
3,7,"In the era of intelligent interconnection, art...",era intelligent interconnection artificial int...,industry
4,8,Artificial intelligence (AI) is a powerful too...,artificial intelligence AI powerful tool techn...,industry
...,...,...,...,...
10980,16779,Unmanned Aerial Vehicles (UAVs) or Unmanned Ai...,aerial vehicle uav unmanned aircraft system ua...,traffic
10981,16780,Background The sustained interest in electroni...,sustain interest electronic assistive technolo...,traffic
10982,16782,Hazardous waste management (HWM) involves tran...,hazardou waste management HWM involve transpor...,waste
10983,16784,IMPORTANCE Cardiac telerehabilitation (CTR) ha...,IMPORTANCE cardiac telerehabilitation CTR safe...,traffic


In [ ]:
print(digital['token'].apply(lambda x:len(str(x).split())).max())
max_len = (round(digital['token'].apply(lambda x:len(str(x).split())).max()/100)*100)+50
print(max_len)

426
450


In [ ]:
xtest = digital.token.values

# using keras tokenizer here
token = text.Tokenizer(num_words=None)
# max_len = 1000

token.fit_on_texts(list(xtest))
xtest_seq = token.texts_to_sequences(xtest)

#zero pad the sequences
xtest_pad = pad_sequences(xtest_seq, maxlen=max_len)

word_index = token.word_index

In [ ]:
ytest_model = model.predict(xtest_pad)
ytest_model

344/344 [==============================] - 10s 25ms/step


array([[5.5016301e-08, 2.7528958e-07, 4.6111438e-07, ..., 4.9708203e-11,
        1.1752666e-06, 3.3827647e-09],
       [9.9999440e-01, 5.2234686e-08, 8.9597471e-08, ..., 2.0795117e-08,
        1.1305184e-06, 2.9308680e-07],
       [6.4565289e-01, 9.0760394e-04, 7.5627514e-04, ..., 1.1617537e-05,
        1.7833686e-03, 7.1496633e-04],
       ...,
       [9.9908173e-01, 3.0710831e-05, 3.3354873e-04, ..., 1.7925855e-05,
        9.8591940e-05, 2.2637744e-04],
       [9.8851210e-01, 8.1272236e-05, 4.7375352e-04, ..., 6.3297157e-05,
        7.3324976e-05, 2.8676700e-04],
       [6.5625710e-03, 4.4681703e-05, 8.9822261e-06, ..., 7.5828523e-04,
        1.4288378e-03, 3.2208062e-04]], dtype=float32)

In [ ]:
print(xtest[0])
print(np.where(ytest_model[0] == np.max(ytest_model[0]))[0].item())

technological advancement information communication technology ict artificial intelligence AI internet-of-thing iot popularity smart city smart movement last couple decade development smart home domain number smart home relate systematic evolution domain study help understanding historical vicissitude state-of-the-art trend smart home cluster study scientometric method analyse publish last decade scientometric finding reveal smart home literature steady growth last decade smart home ICT home automation home information management AI home automation domestic energy management iot home automation home-based healthcare area iot popular technology realise smart home limit urban perspective social issue home technology smart home strong smart city agenda
3


In [ ]:
word

,cate,toxic,class
0,energy,11,0
1,industry,12,1
2,waste,13,2
3,traffic,14,3
4,building,15,4
5,land-use,16,5
6,green,21,6
7,biodiversity,22,7
8,water,23,8
9,heat island,24,9


In [ ]:
predict = {'token': [], 'class': [], 'true_cate': [], 'predict_cate': []}
for i in tqdm(range(len(xtest))):
  predict['token'].append(xtest[i])
  predict['class'].append(np.where(ytest_model[i] == np.max(ytest_model[i]))[0].item())
  predict['true_cate'].append(digital[(digital['token'] == xtest[i])]['category'].values[0])
  predict['predict_cate'].append(word[(word['class'] == np.where(ytest_model[i] == np.max(ytest_model[i]))[0].item())]['cate'].values[0])
predict = pd.DataFrame(predict)
predict

100%|██████████| 10985/10985 [00:29<00:00, 373.78it/s]


,token,class,true_cate,predict_cate
0,technological advancement information communic...,3,energy,traffic
1,internet thing iot hot topic present tech-driv...,0,traffic,energy
2,technological innovation materiality mechanic ...,0,traffic,energy
3,era intelligent interconnection artificial int...,0,industry,energy
4,artificial intelligence AI powerful tool techn...,0,industry,energy
...,...,...,...,...
10980,aerial vehicle uav unmanned aircraft system ua...,8,traffic,water
10981,sustain interest electronic assistive technolo...,2,traffic,waste
10982,hazardou waste management HWM involve transpor...,0,waste,energy
10983,IMPORTANCE cardiac telerehabilitation CTR safe...,0,traffic,energy


In [ ]:
pd.DataFrame(predict).to_excel('0818_digital_predict.xlsx', index=False)